In [1]:
results = {
    "test_brier_0th_event": [0.10976622852935834, 0.10373273544959416,],
    "test_brier_1th_event": [0.09804221797341939, 0.1061306646165277,],
    "test_brier_avg": [0.10390422325138887, 0.10493170003306093],
    "test_ipcw_avg_0th_event": [0.848107387613975, 0.8625205112198908],
    "test_ipcw_avg_1th_event": [0.8359439948707906, 0.8586121838809813,],
    "test_ipcw_avg": [0.8420256912423828, 0.860566347550436],
    "test_eipcw_eventwise_cindex": [0.7228915662650602, 0.7450980392156863],
}
# calculate the average of the results and the std of the results
import numpy as np
def calculate_average_and_std(results):
    averages = {}
    stds = {}
    for key, values in results.items():
        averages[key] = np.mean(values)
        stds[key] = np.std(values)
    return averages, stds

averages, stds = calculate_average_and_std(results)
# print the results
for key in sorted(averages.keys()):
    print(f"{key}: {averages[key]:.4f} ± {stds[key]:.4f}")

test_brier_0th_event: 0.1067 ± 0.0030
test_brier_1th_event: 0.1021 ± 0.0040
test_brier_avg: 0.1044 ± 0.0005
test_eipcw_eventwise_cindex: 0.7340 ± 0.0111
test_ipcw_avg: 0.8513 ± 0.0093
test_ipcw_avg_0th_event: 0.8553 ± 0.0072
test_ipcw_avg_1th_event: 0.8473 ± 0.0113


In [1]:
results = {
    "test_brier_0th_event": [0.10487218199378326, 0.10454063766639302,],
    "test_brier_1th_event": [0.09823392252152408, 0.10061322038790109,],
    "test_brier_avg": [0.10155305225765367, 0.10257692902714705],
    "test_ipcw_avg_0th_event": [0.8467654831932533, 0.8423666285481073],
    "test_ipcw_avg_1th_event": [0.862022782105241, 0.8488232815155962,],
    "test_ipcw_avg": [0.8543941326492471, 0.8455949550318518],
    "test_eipcw_eventwise_cindex": [0.6744186046511628, 0.6210526315789474],
}
# calculate the average of the results and the std of the results
import numpy as np
def calculate_average_and_std(results):
    averages = {}
    stds = {}
    for key, values in results.items():
        averages[key] = np.mean(values)
        stds[key] = np.std(values)
    return averages, stds

averages, stds = calculate_average_and_std(results)
# print the results
for key in sorted(averages.keys()):
    print(f"{key}: {averages[key]:.4f} ± {stds[key]:.4f}")

test_brier_0th_event: 0.1047 ± 0.0002
test_brier_1th_event: 0.0994 ± 0.0012
test_brier_avg: 0.1021 ± 0.0005
test_eipcw_eventwise_cindex: 0.6477 ± 0.0267
test_ipcw_avg: 0.8500 ± 0.0044
test_ipcw_avg_0th_event: 0.8446 ± 0.0022
test_ipcw_avg_1th_event: 0.8554 ± 0.0066


In [3]:
results = {
    "test_brier_0th_event": [0.08573217259337428, 0.08561829126624927,],
    "test_brier_1th_event": [0.08396465102544785, 0.08420357265229417,],
    "test_brier_avg": [0.08484841180941106, 0.08491093195927171],
    "test_ipcw_avg_0th_event": [0.8595055414559984, 0.8466262726656155],
    "test_ipcw_avg_1th_event": [0.8485310044948349, 0.8614636769226476,],
    "test_ipcw_avg": [0.8540182729754167, 0.8540449747941316],
    "test_eipcw_eventwise_cindex": [0.6842105263157895, 0.673469387755102],
}
# calculate the average of the results and the std of the results
import numpy as np
def calculate_average_and_std(results):
    averages = {}
    stds = {}
    for key, values in results.items():
        averages[key] = np.mean(values)
        stds[key] = np.std(values)
    return averages, stds

averages, stds = calculate_average_and_std(results)
# print the results
for key in sorted(averages.keys()):
    print(f"{key}: {averages[key]:.4f} ± {stds[key]:.4f}")

test_brier_0th_event: 0.0857 ± 0.0001
test_brier_1th_event: 0.0841 ± 0.0001
test_brier_avg: 0.0849 ± 0.0000
test_eipcw_eventwise_cindex: 0.6788 ± 0.0054
test_ipcw_avg: 0.8540 ± 0.0000
test_ipcw_avg_0th_event: 0.8531 ± 0.0064
test_ipcw_avg_1th_event: 0.8550 ± 0.0065
